# Описание проекта
**Исследование надёжности заёмщиков**

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откроем файл с данными и изучим общую информацию

Импортируем библиотеки. Взглянем на присланную разработчиками таблицу.

In [ ]:
#импортируем библиотеки pandas и numpy
import pandas as pd
import numpy as np
# импортируем pymystem3
from pymystem3 import Mystem

# если путь к файлу изменится, мы увидим сообщение
try:
    df = pd.read_csv('/datasets/data.csv')
except:
    print('Проверьте абсолютный путь к датасету')

#выведем на экран первый 10 строк датафрейма
display(df.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Ознакомимся с данными, оценим объём данных в таблице df.

In [ ]:
#выведем на экран все атрибуты таблицы
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

Выведем на экран уникальные данные в колонках **children**, **education**, **education_id**, **family_status**, **family_status_id**,
**gender**, **income_type**, **purpose** в таблице `df`.

In [ ]:
# если названия столбцов изменятся мы увидим предупреждение
try:
    display("Уникальные данные в столбце 'children': {}".format(df['children'].unique()))
    display("Уникальные данные в столбце 'education': {}".format(df['education'].unique()))
    display("Уникальные данные в столбце 'education_id': {}".format(df['education_id'].unique()))
    display("Уникальные данные в столбце 'family_status': {}".format(df['family_status'].unique()))
    display("Уникальные данные в столбце 'family_status_id': {}".format(df['family_status_id'].unique()))
    display("Уникальные данные в столбце 'gender': {}".format(df['gender'].unique()))
    display("Уникальные данные в столбце 'income_type': {}".format(df['income_type'].unique()))
    display("Уникальные данные в столбце 'purpose': {}".format(df['purpose'].unique()))
except:
    print('Проверьте название подаваемых столбцов')

Проверьте название подаваемых столбцов


Выведем минимальные и максимальные значения в таблице df

In [ ]:
#ознакомимся с минимальными и максимальными значениями в таблице
print('Минимальные значения:')
display(df.min())

print('Максимальные значения:')
display(df.max())

Минимальные значения:


children                               -1
days_employed                    -18388.9
dob_years                               0
education                          ВЫСШЕЕ
education_id                            0
family_status       Не женат / не замужем
family_status_id                        0
gender                                  F
income_type                   безработный
debt                                    0
total_income                      20667.3
purpose                        автомобили
dtype: object

Максимальные значения:


children                         20
days_employed                401755
dob_years                        75
education            ученая степень
education_id                      4
family_status       женат / замужем
family_status_id                  4
gender                          XNA
income_type                 студент
debt                              1
total_income             2.2656e+06
purpose             сыграть свадьбу
dtype: object

**Вывод**

В таблице 21525 строк, 12 столбцов.

Согласно документации к данным:  
*  **children** — количество детей в семье;  
*  **days_employed** — общий трудовой стаж в днях;  
*  **dob_years** — возраст клиента в годах;  
*  **education** — уровень образования клиента;  
*  **education_id** — идентификатор уровня образования;  
*  **family_status** — семейное положение;  
*  **family_status_id** — идентификатор семейного положения;  
*  **gender** — пол клиента;  
*  **income_type** — тип занятости;  
*  **debt** — имел ли задолженность по возврату кредитов;  
*  **total_income** — ежемесячный доход;  
*  **purpose** — цель получения кредита;  


Выявленные ошибки:
* Количество значений в столбцах различается. Значит, в данных есть пропущенные значения в столбцах **total_income**, **days_employed**;  
* В столбце **purpose** встречаются неявные дубликаты (`автомобиль`, `автомобили`);  
* В столбце **dob_years** встречаются нулевые значения, что не логично;  
* В столбце **days_employed** встречаются как положительные так и отрицательные значения;  
* В столбце **children** встречается отрицательное значение. Аномальное значение 20, значительно отличающееся от остальных значений в этом столбце;  
* В столбце **education** и **family_status** данные записаны как в нижнем так и в верхнем регистре;  
* В столбце **gender** встречаются пропуски `XNA`.

Предварительно можно утверждать, что, данных достаточно для проведения исследования.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

Посчитаем, сколько в таблице пропущенных значений

In [ ]:
# ознакомимся где в таблице встречаются пропущенные значения
print("В таблице df пропущенные значения встречаются в столбцах:")
display(df.isna().sum())

В таблице df пропущенные значения встречаются в столбцах:


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Восстановим значения в столбце **total_income**, заменив значение `NaN` на медианное значение по столбцу **total_income** методом `fillna()`

In [ ]:
#посчитаем медианное значение по столбцу total_income и заменим ими пустые данные
total_income_avg = df['total_income'].median()
df['total_income'] = df['total_income'].fillna(total_income_avg)

#проверим изменения выведя результат на экран
print('После восстановления, пустые значения в столбцах:')
display(df.isna().sum())

После восстановления, пустые значения в столбцах:


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

В столбце **gender** встречаются пропуски `XNA`, заменим их на один тип данных `F`

In [ ]:
# перезапишем значения XNA в столбце gender на значение F
df.loc[df['gender'] == 'XNA', 'gender'] = 'F'

# после этого проверим вывод уникальных значений для столбца gender
display("Уникальные данные в столбце 'gender': {}".format(df['gender'].unique()))

"Уникальные данные в столбце 'gender': ['F' 'M']"

В столбцах **education** и **family_status** данные приведем в нижний регистр

In [ ]:
# перезапишем данные в нижнем регистре данные в столбцах education и family_status
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()

#Проверим вывод
display("Уникальные данные в столбце 'education': {}".format(df['education'].unique()))
display("Уникальные данные в столбце 'family_status': {}".format(df['family_status'].unique()))

"Уникальные данные в столбце 'education': ['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']"

"Уникальные данные в столбце 'family_status': ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'\n 'не женат / не замужем']"

Выведем на экран кол-во значений (`-1,` `20`) встречающихся в столбце **children**, чтобы понимать, можно ли ими принебречь, какое %-ое соотношение к другим значениям в этом столбце.

In [ ]:
# посчитаем какие данные встречаются в столбце children
print('Данные и их количество в столбце children:')
display(df['children'].value_counts())

Данные и их количество в столбце children:


 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Исправим отрицательное значение (-1) и максимальное значение (20) в столбце `children`

In [ ]:
#Вызов метода replace() пришлось повторить дважды. Если бы неправильных написаний было больше,
#- увеличилось бы и число повторов. Не пришлось писать специальную собственную функцию
df['children'] = df['children'].replace(-1, 0)
df['children'] = df['children'].replace(20, 0)

#проверим, сколько теперь уникальных значений в столбце и их количество
print('Данные и их количество в столбце children:')
display(df['children'].value_counts())

Данные и их количество в столбце children:


0    14272
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

**Вывод**

* Обнаружены пропуски в столбцах **days_employed**, **total_income**. Количество значения `NaN` равняется 2174, что составляет `~10%` от количества строк в таблице.
Значения в столбце **days_employed** в исследовании не требуются, в столбце **total_income** заменили пропуски методом `fillna()` взяв медианные значения столбца **total_income**. Возможная причина появления пропусков - ошибка при выгрузке таблицы, работа каких-то методов при конвертации.

* В столбце **gender** обнаружился тип `XNA`. Для исследования данные из этого столбца не требуются, следовательно, мы заменили его на любое значение этого столбца, выбрав `F` в качестве аргумента, применив метод `.loc` для замены строковых значений. Восстановить пол в столбце не представляется возможным, т.к. отсутствует столбец с именем, по которому можно было бы восстановить пропущеннную информацию. Возможная причина появления пропусков - человеческий фактор, человек вносивший информацию в таблицу не указал пол.

* В столбцах **education** и **family_status** данные были написано с применением верхнего и нижнего регистра, привели все значения к нижнему регистру. Учитывая, что написание в верхнем регистре практически полностью дублировало написаное слов в нижнем, ошибка возникла при объединении таблиц, где в одной были названия написано в нижнем регистре, во второй же таблице были написаны в верхнем, при соединении таблиц остались оба варианта написания.

* В столбце **children** встретились два аномальных типа данных принимающих значения `-1` и `20`, выходящие за порядок других значени в этом столбце. Данных, принимающих значения `-1` и `20` в столбце насчитывалось 123 штуки, что составляет порядка `0.5 %` от общего количества данных в столбце. Заменили эти значения на 0, взяв его как наибольшее из представленных. Возможная причина - ошибка при выгрузке таблицы.

### Замена типа данных

Заменим значения в столбцах **days_employed** и **days_employed** с `float64` на `int64`. Предположим, что как будто бы пропуск = никогда не работал.

In [ ]:
# заменим значения NaN в столбце 'days_employed' на 0
df['days_employed'] = df['days_employed'].fillna(value=0)

# переведем float в int
df['total_income'] = df['total_income'].astype('int')
df['days_employed'] = df['days_employed'].astype('int')

#выведем типы данных проверить что все применилось
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Чтобы работать с целыми числами, для оптимизации и ускорения просчетов, мы значения перевели в целые числа методом `.astype('int')`. Метод `to_numeric()` не использовался, т.к. особенность метода  в том, что при переводе все числа будут иметь тип данных float.

### Обработка дубликатов

Произведем поиск дубликатов в датафрейме df

In [ ]:
#выведем на экран кол-во найденных явных дубликатов, метод .sum() возвращает количество дубликатов
print(f'Явных дубликатов: {df.duplicated().sum()}')
display(df[df.duplicated(keep=False)].sort_values(by = 'dob_years', ascending = True).head())

#удалим явные дубликаты и переиндексируем таблицу
df = df.drop_duplicates().reset_index(drop=True)

#проверим количество явных дубликатов после удаления
#display(df[df.duplicated(keep=False)].sort_values(by = 'dob_years', ascending = True).head())
print(f'После обработки, в датафрейме явных дубликатов: {df.duplicated().sum()}')

Явных дубликатов: 71


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
20297,1,0,23,среднее,1,гражданский брак,1,F,сотрудник,0,145017,сыграть свадьбу
8853,1,0,23,среднее,1,гражданский брак,1,F,сотрудник,0,145017,сыграть свадьбу
15892,0,0,23,среднее,1,не женат / не замужем,4,F,сотрудник,0,145017,сделка с подержанным автомобилем
19321,0,0,23,среднее,1,не женат / не замужем,4,F,сотрудник,0,145017,сделка с подержанным автомобилем
3452,0,0,29,высшее,0,женат / замужем,0,M,сотрудник,0,145017,покупка жилой недвижимости


После обработки, в датафрейме явных дубликатов: 0


**Вывод**

Методом `duplicated()` нашли явные дубликаты, методом `drop_duplicates()` избавились от полных дубликатов.
Метод `value_counts()` применять не имеет смысла, значения в столбцах с строковыми данными будут повторяться (должность, цель кредита).  
Возможные причины появления полных дубликатов в данных - данные в датафрейм были склеены из нескольких таблиц.

### Лемматизация

Лемматизируем данные столбца **purpose**

In [ ]:
m = Mystem()

# функция лемматизации строки
def lemmer(row):
    lemmas = ' '.join(m.lemmatize(row['purpose']))
    return lemmas.replace(' \n','')
df['purpose_lem'] = df.apply(lemmer, axis=1)
df.purpose_lem.unique()

array(['покупка   жилье', 'приобретение   автомобиль',
       'дополнительный   образование', 'сыграть   свадьба',
       'операция   с   жилье', 'образование', 'на   проведение   свадьба',
       'покупка   жилье   для   семья', 'покупка   недвижимость',
       'покупка   коммерческий   недвижимость',
       'покупка   жилой   недвижимость',
       'строительство   собственный   недвижимость', 'недвижимость',
       'строительство   недвижимость',
       'на   покупка   подержать   автомобиль',
       'на   покупка   свой   автомобиль',
       'операция   с   коммерческий   недвижимость',
       'строительство   жилой   недвижимость', 'жилье',
       'операция   со   свой   недвижимость', 'автомобиль',
       'заниматься   образование',
       'сделка   с   подержанный   автомобиль', 'получение   образование',
       'свадьба', 'получение   дополнительный   образование',
       'покупка   свой   жилье', 'операция   с   недвижимость',
       'получение   высокий   образование', 'свой  

**Вывод**

В столбце цель кредита записи были сделаны без единого стиля, много категорий было излишними. Необходимо было привести слова к словарной форме, лемматизировать.  
При лемматизизации цели кредита, создали новый столбец **purpose_lem**, посмотрев на который - стали прослеживаться 4 категории на которые берут кредит: `свадьба, автомобиль, недвижимость, образование`. Это будет использовано для категоризации данных.

### Категоризация данных

Категоризируем цели кредита из столбца **purpose_lem**, создадим новый столбец **purpose_cat** где будут храниться категоризированные данные.

In [ ]:
# функция для категоризации по цели кредита
def purp_replace(i):
    if 'свадьба' in i:
        return 'свадьба'
    elif 'автомобиль' in i:
        return 'автомобиль'
    elif 'недвижимость' in i or 'жилье' in i:
        return 'недвижимость'
    elif 'образование' in i:
        return 'образование'
    else:
        return 5

# вызов функции и создание столбца с категоризированными данными, последующий вывод на экран соответствия значений и их кол-ва
df['purpose_cat'] = df['purpose_lem'].apply(purp_replace)
df['purpose_cat'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_cat, dtype: int64

Категоризируем клиентов по наличию детей.

In [ ]:
# функция для категоризации по наличию детей
def count_child(child):
    if child == 0:
        return 'детей нет'
    return 'дети есть'

# вызов функции по категоризации и создание нового столбца в таблице, последующи вывод на экран соответствия значений и их кол-ва
df['child_count'] = df['children'].apply(count_child)
df['child_count'].value_counts()

детей нет    14214
дети есть     7240
Name: child_count, dtype: int64

Категоризируем заемщиков по заработной плате. Основываться будем на `РАСПРЕДЕЛЕНИЕ НАСЕЛЕНИЯ ПО ВЕЛИЧИНЕ СРЕДНЕДУШЕВЫХ ДЕНЕЖНЫХ ДОХОДОВ`

In [ ]:
# категоризация для столбца с уровнем зарплаты
def total_income_func(zarpl):
    if zarpl <= 7000:
        return 'уровень 1'
    if zarpl <= 10000:
        return 'уровень 2'
    if zarpl <= 14000:
        return 'уровень 3'
    if zarpl <= 19000:
        return 'уровень 4'
    if zarpl <= 27000:
        return 'уровень 5'
    if zarpl <= 45000:
        return 'уровень 6'
    if zarpl <= 60000:
        return 'уровень 7'
    if zarpl <= 75000:
        return 'уровень 8'
    if zarpl <= 100000:
        return 'уровень 9'
    return 'уровень 10'

# вызов функции, создание нового столбца total_income_cat, проверка с выводом на экран данных из столбца total_income_cat
df['total_income_cat'] = df['total_income'].apply(total_income_func)
df['total_income_cat'].value_counts()

уровень 10    16991
уровень 9      2598
уровень 8      1059
уровень 7       589
уровень 6       205
уровень 5        12
Name: total_income_cat, dtype: int64

In [ ]:
# функция для категоризации семейного положения
def marriage_count(married):
    if married == 'женат / замужем':
        return 'в браке'
    return 'не в браке'

# вызов функции для создания нового столбца married, вывод на экран значений и их кол-ва
df['married'] = df['family_status'].apply(marriage_count)
df['married'].value_counts()

в браке       12339
не в браке     9115
Name: married, dtype: int64

**Вывод**

Для дальнешего исследования влияния наличия детей, уровня доходов и цель кредита на погашение кредитов - мы произвели категоризацию данных с добавлением новых столбцов в датафрейм. Созданные функции будут использоваться далее при расчете зависомостей.  
- Для категоризации цели заема были выявлены 4 явные направления на что берут кредит
- Семеное положение согласно СК РФ `Признается брак, заключенный только в органах записи актов гражданского состояния`, следовательно категоризировали по записи `женат / замужем`, туда не попадают граждане живущие в гражданском браке.
- Для проверки влияния наличия детей была создана категория делящая заемщиков на 2 типа: дети есть / детей нет
- Для разделения заемщиков по уровню дохода, основываясь на `Показатели  по Российской Федерации рассчитаны с использованием величины макроэкономического показателя среднедушевых денежных доходов населения, определенной в соответствии с Методологическими положениями по расчету показателей денежных  доходов и расходов населения` от Федеральной службы государственной статистики. Была произведена категоризация на 10 уровней.   
[приказ Росстата от 2 июля 2014 года  № 465 с изменениями от 20 ноября 2018 года](https://rosstat.gov.ru/folder/13397/)

Категоризация произведена по следующему принципу:
- Уровень 1 - заемщики с уровнем дохода до 7000 руб.
- Уровень 2 - заемщики с уровнем дохода 7000 - 10000 руб.
- Уровень 3 - заемщики с уровнем дохода 10000 - 14000 руб.
- Уровень 4 - заемщики с уровнем дохода 14000 - 19000 руб.
- Уровень 5 - заемщики с уровнем дохода 19000 - 27000 руб.
- Уровень 6 - заемщики с уровнем дохода 27000 - 45000 руб.
- Уровень 7 - заемщики с уровнем дохода 45000 - 60000 руб.
- Уровень 8 - заемщики с уровнем дохода 60000 - 75000 руб.
- Уровень 9 - заемщики с уровнем дохода 75000 - 100000 руб.
- Уровень 10 - заемщики с уровнем дохода свыше 100000 руб.  

## Шаг 3. Ответим на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
# визуализируем вывод таблицы зависимость наличия детей и имел ли задолженности по кредиту
children_to_debt = df.pivot_table(index = ['child_count'], values = 'debt')*100
display(children_to_debt)

,debt
child_count,
детей нет,7.541860
дети есть,9.240331


**Вывод**

Клиенты, у которых `есть дети`, чаще имели задолженность по возврату кредитов, чем клиенты, у которых `нет детей`. Таким образом теория, что зависимость между наличием детей и возвратом кредита в срок - подтвердилась.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
df_pivot_married_level = df.pivot_table(index=['married'], columns='debt', values='family_status_id', aggfunc='count')
df_pivot_married_level['married_debt_percent'] = df_pivot_married_level[1]/df_pivot_married_level[0]*100
display(df_pivot_married_level.sort_values(by = 'married_debt_percent', ascending = True))

debt,0,1,married_debt_percent
married,,,
в браке,11408,931,8.160940
не в браке,8305,810,9.753161


**Вывод**

Зависимость между семейным положением и возвратом кредита в срок есть: клиенты указавшие в заявке `женат / замужем` - чаще возвращают кредиты в срок. Клиенты объединенные в группу `не в браке`, записавшие данные при подаче как `'гражданский брак' 'вдовец / вдова' 'в разводе' 'не женат / не замужем'` чаще склонны к просрочке кредита.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
df_pivot_zarpl_to_debt = df.pivot_table(index=['total_income_cat'], columns='debt', values='family_status_id', aggfunc='count')
df_pivot_zarpl_to_debt['total_income_debt_percent'] = df_pivot_zarpl_to_debt[1]/df_pivot_zarpl_to_debt[0]*100
display(df_pivot_zarpl_to_debt.sort_values(by = 'total_income_debt_percent', ascending = True))

debt,0,1,total_income_debt_percent
total_income_cat,,,
уровень 7,554,35,6.317690
уровень 6,192,13,6.770833
уровень 10,15604,1387,8.888746
уровень 8,972,87,8.950617
уровень 5,11,1,9.090909
уровень 9,2380,218,9.159664


**Вывод**

Исследование подтвердило теорию что зависимость между уровнем дохода и возвратом кредита в срок - есть. Подтверждением служит таблица выше, из которой следует что заемщики, указавшие уровень зарплат `7, 6` - чаще возвращали кредит в срок, в то время как заемщики с уровнем зарплат `10, 8, 5 и 9` - чаще других имели задолженность по возврату кредита. Интересная особенность, категория `10`, с самым высоким заработком, находятся в середине списка, значит высокий уровень дохода не гарантирует возврат кредита в срок. Еще отметим, что категорий заемщиков с уровнем `1, 2, 3, 4` - в выгрузке датафрейма не обнаружены.

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
df_pivot_purpose_to_debt = df.pivot_table(index=['purpose_cat'], columns='debt', values='family_status_id', aggfunc='count')
df_pivot_purpose_to_debt['purpose_to_debt_percent'] = df_pivot_purpose_to_debt[1]/df_pivot_purpose_to_debt[0]*100
display(df_pivot_purpose_to_debt.sort_values(by = 'purpose_to_debt_percent', ascending = True))

debt,0,1,purpose_to_debt_percent
purpose_cat,,,
недвижимость,10029,782,7.797388
свадьба,2138,186,8.699719
образование,3643,370,10.156464
автомобиль,3903,403,10.325391


**Вывод**

Можно сделать вывод что заемщики, берущие кредит на `недвижимость` - чаще его возвращали. Далее в порядке увеличения количества  не возвратов идет `свадьба`, `образование`, и больше всего имели задолженность по возврату кредитов клиенты указавшие целью `автомобиль`.

## Шаг 4. Общий вывод

Применяя написанные выше функции мы выяснили как категоризированные данные влияют на факт погашения кредита в срок.
Обобщить информацию можно так:
- самый `благоприятный заемщик` человек состоящий `в браке, без детей, с уровнем зарплаты 27000-60000 рублей, с целью получения кредита на недвижимость`;
- `неблагонадежный заемщик`, это человек у которого `есть дети, не состоит в браке, уровень зарплаты 75000-100000 рублей, берущий кредит с целью на образование или автомобиль`.  
  
В дальнейшем можно легко продолжить исследования для построения кредитного скоринга, добавив несколько функций по категоризации, возможно добавление в скоринг возраста, пола заемщика - расширит функционал, мы сможем получить новые данные, но на этом этапе этого не требовалось.